In [ ]:
!pip install twitter 
!pip install NetworkX
!pip install powerlaw
!pip install string
!pip install textblob

In [ ]:
import powerlaw
import pandas as pd
from textblob import TextBlob
import requests
import pickle
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
import networkx as nx
import string
import re
from itertools import combinations

nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
df = pd.read_csv('df_tutti.csv')

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation.replace('#','')])
    #text = re.sub('[0-9]+', '', text)
    return text

def remove_hashtags(text):
    # Define the regular expression pattern to match words starting with "#"
    pattern = re.compile(r"\B#\w\w+\b\s*")

    # Use the sub() function to replace matched patterns with an empty string
    result = pattern.sub("", text)

    return result

def remove_useless_words(text):
    stop_words = set(stopwords.words('italian'))

def combina(text):
    comb=list(combinations(text, 2))
    return comb

#rimuoviamo dai tweet punteggiatura, hashtags e i tweet senza hashtags
    
df=df.drop_duplicates('tweet', keep='first')
df['Tweet_punct'] = df['tweet'].apply(lambda x: remove_punct(x))

df['Tweet_punct']=df['Tweet_punct'].apply(lambda x: remove_hashtags(x))


df['#'] = df['hashtags'].apply(lambda x: remove_punct(x))

df['#'] = df['#'].apply(lambda x: 'none' if len(x) == 0 else x)
    
#mask = df['#'].str.len() > 0
#df = df[mask]


#rimuoviamo dai tweet parole inutili e inferiori a lunghezza 3

stop_words = set(stopwords.words('italian'))

tweets = [tweet.lower().split() for tweet in df['Tweet_punct']]
tags = [t.lower().split() for t in df['#']]
tweets = [[word for word in tweet_words if not word in stop_words] for tweet_words in tweets]
tweets = [[word for word in tweet_words if len(word) >3] for tweet_words in tweets]


#creiamo dataframe (df_bigram_counts) con qualsiasi bigramma in un tweet e relativo counts tra i tweet
for i in range(0,len(tweets)):
    if i==0:
        comb=list(combinations(tweets[i], 2))
    else:
        comb[len(comb):]=list(combinations(tweets[i], 2))
bigram_counts = collections.Counter(comb)

df_bigram_counts = pd.DataFrame(bigram_counts.most_common(len(bigram_counts)), columns=['bigram', 'count'])


#creiamo dataframe (df_bigram_tags)con bigramma e tutti i relativi hashtag tra i tweet

comb_tag_df2=pd.DataFrame()
comb_tag_df2['words']=tweets
comb_tag_df2['tags']=tags

combina=[]
all_t=[]
for w, t in comb_tag_df2.values:
    for bi in list(combinations(w,2)):
        combina.append(bi)
        all_t.append(t)
df_bigram_tags=pd.DataFrame()
df_bigram_tags['bigram']=combina
df_bigram_tags['tags']=all_t
df_bigram_tags=df_bigram_tags.groupby(['bigram']).sum()


#dataframe finale ordinato per i counts con bigramma e tags, togliamo i bigrammi comparsi in un solo tweet
df_final=pd.merge(left=df_bigram_tags, right=df_bigram_counts, on='bigram')
df_final.sort_values(by=['count'], ascending=False, inplace=True)

mask = df_final['count'] > 1
df_final = df_final[mask]

df_final.reset_index(inplace=True, drop=True)

def group_none(lst):
    new_lst = []
    none_count = 0
    for item in lst:
        if item == 'none':
            none_count += 1
        else:
            new_lst.append(item)
    if none_count > 0:
        new_lst.append(f"{none_count}none")
    return new_lst

# tutti i tag None li riaggruppa specificando quanti sono
df_final['tags'] = df_final['tags'].apply(group_none)

df_final.to_csv('dataset_processato.csv', index=False)